<a href="https://colab.research.google.com/github/chermi22/alx-low_level_programming/blob/master/streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit pdfplumber pandas scikit-learn sentence-transformers faiss-cpu transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 106.6 MB/s eta 0:00:00


In [ ]:
streamlit_code = """
import streamlit as st
import pdfplumber
import re
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline
import numpy as np

st.title("PDF Chatbot")

uploaded_files = st.file_uploader("Upload PDF files", type="pdf", accept_multiple_files=True)

if uploaded_files:
    st.success(f"{len(uploaded_files)} files uploaded")

    documents = []
    filenames = []

    for file in uploaded_files:
        text = ""
        with pdfplumber.open(file) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + " "
        text = re.sub(r'\\n+', ' ', text).lower()
        documents.append(text)
        filenames.append(file.name)

    chunks = []
    chunk_size = 500
    for i, doc in enumerate(documents):
        words = doc.split()
        for j in range(0, len(words), chunk_size):
            chunk_text = " ".join(words[j:j+chunk_size])
            chunks.append({"filename": filenames[i], "text": chunk_text})

    model = SentenceTransformer('all-MiniLM-L6-v2')
    texts = [chunk['text'] for chunk in chunks]
    embeddings = model.encode(texts, show_progress_bar=True)
    embeddings = np.array(embeddings)

    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)

    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

    def search(query, top_k=3):
        k = min(top_k, len(chunks))
        query_vec = model.encode([query])
        distances, indices = index.search(np.array(query_vec), k)
        results = [chunks[idx]["text"] for idx in indices[0]]
        return results

    st.subheader("Ask a question")
    user_question = st.text_input("Your question:")

    if user_question:
        relevant_chunks = search(user_question, top_k=3)
        combined_text = " ".join(relevant_chunks)
        try:
            summary = summarizer(combined_text, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
        except:
            summary = combined_text
        st.markdown("**Answer:**")
        st.write(summary)
"""


In [ ]:
with open("app.py", "w") as f:
    f.write(streamlit_code)

print("✅ Streamlit app saved as app.py")


✅ Streamlit app saved as app.py


In [ ]:
!pip install pyngrok

from pyngrok import ngrok

# Run Streamlit in background
get_ipython().system_raw("streamlit run app.py &")

# Create public URL
public_url = ngrok.connect(port=8501)
print("Open your chatbot here:", public_url)


ERROR:pyngrok.process.ngrok:t=2025-09-17T08:18:21+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-09-17T08:18:21+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-09-17T08:18:21+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.